# 导入包

In [1]:
import pandas as pd
import random
from pyecharts import options as opts
from pyecharts.charts import Line, Bar, Grid,Candlestick,Kline
from pyecharts.globals import CurrentConfig, NotebookType
from pyecharts.commons.utils import JsCode

## 测试数据获取：

In [2]:
o_data = pd.read_csv('test.csv',encoding="GBK" )

In [3]:
o_data

股票代码       交易日期    开盘价    最高价    最低价    收盘价       成交量
0    sh600585   2023/8/9  26.00  26.12  25.76  26.02  15028674
1    sh600585  2023/8/10  26.00  26.16  25.72  25.91  12268763
2    sh600585  2023/8/11  25.94  26.09  25.30  25.35  15609444
3    sh600585  2023/8/14  25.21  25.61  24.76  25.36  18070699
4    sh600585  2023/8/15  25.36  25.64  24.97  25.37  13581230
..        ...        ...    ...    ...    ...    ...       ...
195  sh600585   2024/6/3  24.00  24.48  23.78  24.48  22076561
196  sh600585   2024/6/4  24.25  24.74  24.00  24.56  15303206
197  sh600585   2024/6/5  24.53  24.78  24.30  24.35  12110124
198  sh600585   2024/6/6  24.33  24.48  24.01  24.27  12768293
199  sh600585   2024/6/7  24.20  24.42  24.08  24.30  11696582

[200 rows x 7 columns]

In [4]:
class stock_pandas_to_chart:
    def __new__(cls, data: pd.DataFrame):
        instance = super(stock_pandas_to_chart, cls).__new__(cls)
        instance.__init__(data)
        return instance.grid
    
    
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.date_data = data['交易日期'].tolist()
        self.kindle_data = self.__getkindlelist()
        self.vol_data = data['成交量'].tolist()
        self.macd_data = self.__getmacddata(data['收盘价'].tolist())
        self.grid = None
        
        self.__getkindlelist()
        self.__grid_generate()
    
    def __getkindlelist(self):
        tmplist = []
        for index, row in self.data.iterrows():
            tmplist.append(list(row[['开盘价', '收盘价', '最低价', '最高价']].values))
        return tmplist

    def __getmacddata(self,close_price):
        ema12 = close_price[:1]
        ema26 = close_price[:1]
        for close_p in close_price[1:]:
            ema12new = (ema12[-1]*11/13)+(close_p*2/13)
            ema26new = (ema26[-1]*25/27)+(close_p*2/27)
            ema12.append(ema12new)
            ema26.append(ema26new)
        dif = list(map(lambda a, b: a - b, ema12, ema26))
        dea = dif[:1]
        for each in dif[1:]:
            new = (dea[-1]*0.8)+(each*0.2)
            dea.append(new)
        tmp = list(map(lambda a, b: 2*(a - b), dif, dea))
        macdbar = [round(num, 2) for num in tmp]        
        return macdbar
            
            
    def __grid_generate(self):
        self.grid = Grid()
        self.grid.add( self.__kindle_chart(), grid_opts=opts.GridOpts(pos_bottom="50%"))
        self.grid.add( self.__vol_bar_chart(), grid_opts=opts.GridOpts(pos_top="55%",pos_bottom="30%"))
        self.grid.add( self.__macd_bar_chart(),grid_opts=opts.GridOpts(pos_top="75%"))
    
    def __kindle_chart(self):
        kindle_line = Kline()
        kindle_line.add_xaxis(self.date_data)
        kindle_line.add_yaxis("股票走势", self.kindle_data)
        kindle_line.set_global_opts(
                    title_opts=opts.TitleOpts(title="Stock Price"),
                    xaxis_opts=opts.AxisOpts(type_="category", is_scale=True),
                    yaxis_opts=opts.AxisOpts(is_scale=True),
                    tooltip_opts=opts.TooltipOpts(trigger="axis"),  # 鼠标悬停显示浮窗
                    datazoom_opts=[
                        opts.DataZoomOpts(type_="inside", xaxis_index=[0, 1, 2]),  # 同步到两个图表
                        opts.DataZoomOpts(type_="slider", xaxis_index=[0, 1, 2])   # 同步到两个图表
                    ]
        )
        return kindle_line
    
    def __vol_bar_chart(self):
        vol_bar = Bar()
        vol_bar.add_xaxis(self.date_data)
        vol_bar.add_yaxis(
            "Volume", self.vol_data, yaxis_index=1, color="green", label_opts=opts.LabelOpts(is_show=False),
                        itemstyle_opts=opts.ItemStyleOpts(
                            color=JsCode(
                                """
                                    function(params) {
                                        var colorList;
                                        colorList = '#4EA9F0';
                                        return colorList;
                                    }
                                    """))  
                         )
        vol_bar.set_global_opts(
                xaxis_opts=opts.AxisOpts(type_="category", is_scale=True,axislabel_opts=opts.LabelOpts(is_show=False)),
                yaxis_opts=opts.AxisOpts(
                    is_scale=True,
                    splitline_opts=opts.SplitLineOpts(is_show=True),
                    position="right",
                    axislabel_opts=opts.LabelOpts(is_show=False)  # 隐藏Y轴刻度
                ),
                datazoom_opts=[
                    opts.DataZoomOpts(type_="inside", xaxis_index=[0, 1, 2]),  # 同步到两个图表
                    opts.DataZoomOpts(type_="slider", xaxis_index=[0, 1, 2])   # 同步到两个图表
                ],
                legend_opts=opts.LegendOpts(is_show=False),  # 隐藏图例
            )
        return vol_bar
    

    def __macd_bar_chart(self):
        macd_bar = Bar()
        macd_bar.add_xaxis(self.date_data)
        macd_bar.add_yaxis(
            "macd", self.macd_data, yaxis_index=1, color="green", label_opts=opts.LabelOpts(is_show=False),
            itemstyle_opts=opts.ItemStyleOpts(
                            color=JsCode(
                                """
                                    function(params) {
                                        var colorList;
                                        if (params.data >= 0) {
                                          colorList = '#ef232a';
                                        } else {
                                          colorList = '#14b143';
                                        }
                                        return colorList;
                                    }
                                    """))        
        )
        macd_bar.set_global_opts(
                xaxis_opts=opts.AxisOpts(type_="category", is_scale=True,axislabel_opts=opts.LabelOpts(is_show=False)),
                yaxis_opts=opts.AxisOpts(
                    is_scale=True,
                    splitline_opts=opts.SplitLineOpts(is_show=True),
                    position="right",
                    axislabel_opts=opts.LabelOpts(is_show=False)  # 隐藏Y轴刻度
                ),
                datazoom_opts=[
                    opts.DataZoomOpts(type_="inside", xaxis_index=[0, 1, 2]),  # 同步到两个图表
                    opts.DataZoomOpts(type_="slider", xaxis_index=[0, 1, 2])   # 同步到两个图表
                ],
                legend_opts=opts.LegendOpts(is_show=False),  # 隐藏图例
            ) 
        return macd_bar

In [5]:
# 实例化 PandasChart
c3 = stock_pandas_to_chart(o_data)

#k = c1.kindle_chart()

In [6]:
c3.render_notebook()